In [1]:
from src.data._dataobject import DataObject
from src.data.types._date import NSDate
from src.data.types._datetime import DateTime
from src.data.types._text import String, NSString
from src.data.types._int import Int
from src.data.types._phone_number import PhoneNumber
from src.data.types._id import MongoId, Id, StrId
from src.data.types._time import Time
from src.data.types._nested import Nested
from src.data.types._list import List
from src.connections import Elasticsearch
from src.api._api import Api

In [2]:
class City(DataObject):
    __conn__ = Elasticsearch
    id = StrId(exclude=True)
    name = NSString()
    country_code = NSString(min_length=2, max_length=3, uppercase=True)



class User(DataObject):
    __conn__ = Elasticsearch
    first_name = NSString(min_length=3, json_field='firstName')
    last_name = NSString(json_field='lastName')
    mobile = PhoneNumber(indexable=False)
    id = StrId()
    created = DateTime(default='now', indexable=False)
    birthday = NSDate()
    favourite_time = Time(indexable=False)
    cities = List(City, filterable=False)

User.set_permission('delete', 'admin')


In [3]:
User.drop_table()

True

In [4]:

User.to_es_mapping()

{'mappings': {'properties': {'birthday': {'type': 'date', 'index': True},
   'cities': {'type': 'nested'},
   'created': {'type': 'date', 'index': False},
   'favourite_time': {'type': 'text', 'index': False},
   'first_name': {'type': 'text', 'index': True},
   'last_name': {'type': 'text', 'index': True},
   'mobile': {'type': 'text', 'index': False}}}}

In [5]:
User.__permissions__

{'GET': 'public', 'POST': 'public', 'PUT': 'public', 'DELETE': 'admin'}

In [6]:
User.create_table()

True

In [7]:
u = User(
    first_name='Frodo',
    last_name='Baggins',
    mobile='+39 375 57 44 512',
    birthday='1988-07-25',
    favourite_time='18:00',
    cities=[City(name='Piacenza', country_code='IT')]
)

In [8]:
u.validate()

In [9]:
u.add_to_db()

True

In [10]:
u.to_json(False)

{'birthday': '1988-07-25',
 'cities': [{'country_code': 'IT', '_id': None, 'name': 'Piacenza'}],
 'created': '2022-02-20T11:47:13.489904',
 'favourite_time': '18:00:00.000000',
 'firstName': 'Frodo',
 '_id': 'Jwe-Fn8BZ6AV7k-3PdiQ',
 'lastName': 'Baggins',
 'mobile': '+39 375 57 44 512'}

In [11]:
u.first_name = 'Frodo Jr.'

In [12]:
u.add_to_db()

True

In [13]:
u.to_dict()

{'birthday': '1988-07-25',
 'cities': [{'country_code': 'IT', '_id': None, 'name': 'Piacenza'}],
 'created': '2022-02-20T11:47:13.489904',
 'favourite_time': '18:00:00.000000',
 'first_name': 'Frodo Jr.',
 '_id': 'Jwe-Fn8BZ6AV7k-3PdiQ',
 'last_name': 'Baggins',
 'mobile': '+39 375 57 44 512'}

In [14]:
users = User.get_from_db()

In [15]:
for user in users:
    print(user.to_json())

{"birthday": "1988-07-25", "cities": [{"country_code": "IT", "_id": null, "name": "Piacenza"}], "created": "2022-02-20T11:47:13.489904", "favourite_time": "18:00:00.000000", "firstName": "Frodo", "_id": "Jwe-Fn8BZ6AV7k-3PdiQ", "lastName": "Baggins", "mobile": "+39 375 57 44 512"}


In [16]:
api = Api()
api.add_entity(User)
api.add_entity(City)

In [17]:
# with api:
#     while True:
#         try:
#             continue
#         except:
#             api.stop()
#             break

In [18]:
from psycopg2 import connect
from psycopg2._psycopg import connection

In [19]:
from src.connections import Postgres

In [20]:
with Postgres() as p:
    p.is_connected

In [21]:
p.is_connected

False

In [2]:
class User(DataObject):
    __conn__ = Elasticsearch
    first_name = NSString(min_length=3, json_field='firstName')
    last_name = NSString(json_field='lastName')
    mobile = PhoneNumber(indexable=False)
    id = Id()
    created = DateTime(default='now', indexable=False)
    birthday = NSDate()
    favourite_time = Time(indexable=False)

In [3]:
from src.connections import Sql, Postgres

_tbl = False
with Sql() as alch:
    _tbl = alch.create_table(User)

print(_tbl)

u = User(first_name='John', last_name='Doe', birthday='1993-06-06')

with Postgres() as p:
    print(list(p.select(User, where=[User.birthday >= '1990-01-01'])))

True
[<User id=3>, <User id=4>, <User id=5>]
